In [133]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier # wrapper to use function from sklearn

from tensorflow.keras import layers, Sequential
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV #training and testing data split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:\Projects\Python\Titanic\data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import plotting libraries
import matplotlib.pyplot as plt
#import sns for better plots, it is handy to manage subplots
import seaborn as sns

C:\Projects\Python\Titanic\data\test.csv
C:\Projects\Python\Titanic\data\train.csv


In [134]:
data = pd.read_csv("C:/Projects/Python/titanic/data/train.csv")
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [135]:
# get mean aged based on titel
data['Initial']=0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations

data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

## Assigning the NaN Values with the Ceil values of the mean ages
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']=46
data.Age.isnull().any() #So no null values left finally 

# drop some irrelevant information
droplist = ['PassengerId', 'Name', 'Cabin', 'Ticket']
data.drop(droplist,axis=1,inplace=True)

# one hot encoding of categorical values
ohe_initial = pd.get_dummies(data['Initial'], prefix='title')
ohe_embarked = pd.get_dummies(data['Embarked'], prefix='embarked')
# turn sex into integers instead of string
data['Sex'] = data['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

new_df = pd.merge(data, ohe_initial, left_index=True, right_index=True)
final_df = pd.merge(new_df, ohe_embarked, left_index=True, right_index=True)
droplist = ['Initial', 'Embarked']
final_df.drop(droplist,axis=1,inplace=True)

print(final_df)

#================================= end data preprocessing ================================

     Survived  Pclass  Sex   Age  SibSp  Parch     Fare  title_Master  \
0           0       3    1  22.0      1      0   7.2500             0   
1           1       1    0  38.0      1      0  71.2833             0   
2           1       3    0  26.0      0      0   7.9250             0   
3           1       1    0  35.0      1      0  53.1000             0   
4           0       3    1  35.0      0      0   8.0500             0   
..        ...     ...  ...   ...    ...    ...      ...           ...   
886         0       2    1  27.0      0      0  13.0000             0   
887         1       1    0  19.0      0      0  30.0000             0   
888         0       3    0  22.0      1      2  23.4500             0   
889         1       1    1  26.0      0      0  30.0000             0   
890         0       3    1  32.0      0      0   7.7500             0   

     title_Miss  title_Mr  title_Mrs  title_Other  embarked_C  embarked_Q  \
0             0         1          0          

In [136]:
#========================= start data prep
train, test= train_test_split(final_df, test_size=0.15, stratify=final_df["Survived"])

Xtrain = train[train.columns[1:]]
ytrain = train[train.columns[:1]]
Xtest = test[test.columns[1:]]
ytest = test[test.columns[:1]]

# for cross validation we need to take the complete dataset and pass it as it takes care of test train split
X = final_df[final_df.columns[1:]]
y = final_df[final_df.columns[:1]]

scaler = StandardScaler()
X_normed = scaler.fit_transform(X)

xtrain_val = Xtrain.values
print("xtrain shape before: " + str(xtrain_val.shape))
#xtrain_val = np.reshape(xtrain_val, (-1, 757))
print("xtrain shape after: " + str(xtrain_val.shape))
ytrain_val = ytrain.values
#ytrain_val = np.reshape(ytrain_val, (-1, 757))

xtest_val = Xtest.values
print(xtest_val.shape)
#xtest_val = np.reshape(xtest_val, (-1, 134))
ytest_val = ytest.values
#ytest_val = np.reshape(ytest_val, (-1, 134))

print(xtrain_val.shape)
print(ytrain_val.shape)
print(xtest_val.shape)
print(ytest_val.shape)

xtrain_normed = scaler.fit_transform(xtrain_val)
xtest_normed = scaler.fit_transform(xtest_val)
print(ytest_val)

xtrain shape before: (757, 14)
xtrain shape after: (757, 14)
(134, 14)
(757, 14)
(757, 1)
(134, 14)
(134, 1)
[[1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]]


In [138]:
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(layers.Dense(196, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    #model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    #model.add(layers.Dropout(0.2))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

print(xtest_normed.shape)
print(ytest_val.shape)

#model.fit(xtrain_normed, ytrain_val, validation_data=(xtest_normed, ytest_val), epochs=50, batch_size=4, verbose=2)
sk_model = KerasClassifier(build_fn=create_model, verbose=0)

#scores = cross_val_score(sk_model, X_normed, y, cv=k_fold, scoring='accuracy')
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

#batch_size = [4, 8, 16, 24, 32, 64]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
batch_size = [64, 74, 90, 128]
epochs = [3, 5, 8, 10, 12]
parameter_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)
#Best: 0.834969 using {'batch_size': 90, 'epochs': 5, 'optimizer': 'Nadam'}

gs = GridSearchCV(sk_model, parameter_grid, cv=k_fold, scoring='accuracy', n_jobs=6)
grid_result = gs.fit(X_normed, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

(134, 14)
(134, 1)
Best: 0.834969 using {'batch_size': 90, 'epochs': 5, 'optimizer': 'Nadam'}


In [139]:
# refit the model using the optimal parameters
opti_model = create_model('Nadam')
opti_model.fit(X_normed, y, batch_size=90, epochs=5, shuffle=True)

Epoch 1/5
10/10 [==============================] - 0s 554us/step - loss: 0.6512 - accuracy: 0.7143
Epoch 2/5
10/10 [==============================] - 0s 554us/step - loss: 0.5208 - accuracy: 0.8043
Epoch 3/5
10/10 [==============================] - 0s 554us/step - loss: 0.4538 - accuracy: 0.8257
Epoch 4/5
10/10 [==============================] - 0s 665us/step - loss: 0.4197 - accuracy: 0.8346
Epoch 5/5
10/10 [==============================] - 0s 554us/step - loss: 0.3803 - accuracy: 0.8492


In [92]:
xgb_dmat = xgb.DMatrix(data=xtrain_normed,label=ytrain_val)
xgb_model = xgb.XGBClassifier()
xgb_model.fit(xtrain_normed, ytrain_val)
preds = xgb_model.predict(xtest_normed)
preds = [round(value) for value in preds]
# evaluate predictions
accuracy = accuracy_score(ytest_val, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[13:46:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 76.12%


c:\users\kenneth\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\kenneth\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [140]:
# ==================== apply model
test_data = pd.read_csv("C:/Projects/Python/titanic/data/test.csv")

print(test_data.head())

# get mean aged based on titel
test_data['Initial']=0
for i in data:
    test_data['Initial']=test_data.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations

test_data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Dona','Jonkheer','Col','Rev','Capt','Sir','Don'],
                    ['Mrs','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)


#print(test_data.loc[(test_data.Initial=='Other'),'Age'].mean())

## Assigning the NaN Values with the Ceil values of the mean ages
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Mr'),'Age']=32.1
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Mrs'),'Age']=38.9
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Master'),'Age']=7.4
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Miss'),'Age']=21.8
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Other'),'Age']=42
print(test_data.loc[test_data.Age.isnull()]) #So no null values left finally 

# drop some irrelevant information
droplist = ['PassengerId', 'Name', 'Cabin', 'Ticket']
test_data.drop(droplist,axis=1,inplace=True)

# one hot encoding of categorical values
ohe_initial = pd.get_dummies(test_data['Initial'], prefix='title')
ohe_embarked = pd.get_dummies(test_data['Embarked'], prefix='embarked')
# turn sex into integers instead of string
test_data['Sex'] = test_data['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

new_df = pd.merge(test_data, ohe_initial, left_index=True, right_index=True)
final_test_df = pd.merge(new_df, ohe_embarked, left_index=True, right_index=True)
droplist = ['Initial', 'Embarked']
final_test_df.drop(droplist,axis=1,inplace=True)
print(final_test_df)



   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  
Empty DataFrame
Columns: [PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Initial]
Index: []
     Pclass  Sex   Age  SibSp  Parch      Fare  ti

In [142]:
# data prep
Xtest = final_test_df

print(Xtest.head())
#print(ytest.head())

scaler = StandardScaler()
X_normed = scaler.fit_transform(Xtest)

ypred = opti_model.predict(X_normed)
print(ypred.round())

   Pclass  Sex   Age  SibSp  Parch     Fare  title_Master  title_Miss  \
0       3    1  34.5      0      0   7.8292             0           0   
1       3    0  47.0      1      0   7.0000             0           0   
2       2    1  62.0      0      0   9.6875             0           0   
3       3    1  27.0      0      0   8.6625             0           0   
4       3    0  22.0      1      1  12.2875             0           0   

   title_Mr  title_Mrs  title_Other  embarked_C  embarked_Q  embarked_S  
0         1          0            0           0           1           0  
1         0          1            0           0           0           1  
2         1          0            0           0           1           0  
3         1          0            0           0           0           1  
4         0          1            0           0           0           1  
[[ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.